# Content Based Filtering : Rekomendasi Studi Independen dan Magang Kampus Merdeka

In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_csv("https://raw.githubusercontent.com/adisamatulia/Tugas3_Systemrekomendasi/main/Tugas3_System_Rekomendasi%20-%20Sheet1.csv")
df.head()

,Nama Perusahaan,Alamat,Modul Pembelajaran
0,Adis Amatulia,Kendal,"Text Mining,Grafika Komputer,Sistem Infomasi G..."
1,PT Orbit Ventura Indonesia,"Veteran RI Building 15th Floor Unit Z15-002, P...","Siklus Project Al,Sasaran Pengembangan Skill,"
2,PT Hacktivate Teknologi Indonesia,"Gedung Aquarius Lt. 1&2, Jl. Sultan Iskandar M...",Memahami serta mengimplementasikan library hin...
3,PT Ruang Raya Indonesia,"Jl. Dr. Saharjo No.161, Manggarai Selatan, Teb...","Final Project,software engineering : menerapka..."
4,PT BISA ARTIFISIAL INDONESIA,JLN. IR H JUANDA NO. 84 BANDUNG,Mengenal teknik dan langkah dalam manajemen pr...


## 1. Ikhtisar

In [2]:
df.describe()

,Nama Perusahaan,Alamat,Modul Pembelajaran
count,12,12,12
unique,12,12,12
top,Adis Amatulia,Kendal,"Text Mining,Grafika Komputer,Sistem Infomasi G..."
freq,1,1,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Nama Perusahaan     12 non-null     object
 1   Alamat              12 non-null     object
 2   Modul Pembelajaran  12 non-null     object
dtypes: object(3)
memory usage: 416.0+ bytes


## 2. Modul Pembelajaran (Sebelum Preprocessing)

In [4]:
def print_description(index):
    example = df[df.index == index][['Modul Pembelajaran', 'Nama Perusahaan','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama Perusahaan :', example[1])
        print('Alamat:', example[2])

In [5]:
print_description(0)

Text Mining,Grafika Komputer,Sistem Infomasi Geografis,Statistika,Web Servis 
Nama Perusahaan : Adis Amatulia 
Alamat: Kendal


In [30]:
print_description(4)

Mengenal teknik dan langkah dalam manajemen proyek, mengenal teknik dan langkah dalam manajemen proyek perangkat lunak,Mampu mengenal beragam proyek kecerdasan artifisial, mengenal industri AI,Mampu mengenal cara membuat digital startup dan mengurus aspek legal perusahaan
Nama Perusahaan : PT BISA ARTIFISIAL INDONESIA
Alamat: JLN. IR H JUANDA NO. 84 BANDUNG


In [31]:
print_description(11)

Mampu menerapkan proses desain ke bentuk engineering desain 3D dan drafting sebelum manufaktur,Peserta magang diharapkan dapat melakukan proses desain keteknikan mulai dari merumuskan masalah, menyusun solusi terkait permasalahan teknis, dan menghasilkan desain sistem dengan ide-ide baru di bidang keteknikan.
Nama Perusahaan : PT Inamas Sintesis Teknologi
Alamat: Jl. Werkudara 4, Kel. Wirobrajan, Kec. Wirobrajan, Yogyakarta, D.I.Yogyakarta 55252


## 3. Text Preprocessing

In [32]:
import nltk
nltk.download('stopwords')
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
#stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['modulpembelajaran_clean'] = df['Modul Pembelajaran'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
df.head()

,Nama Perusahaan,Alamat,Modul Pembelajaran,modulpembelajaran_clean
0,Adis Amatulia,Kendal,"Text Mining,Grafika Komputer,Sistem Infomasi G...",text mining grafika komputer sistem infomasi g...
1,PT Orbit Ventura Indonesia,"Veteran RI Building 15th Floor Unit Z15-002, P...","Siklus Project Al,Sasaran Pengembangan Skill,",siklus project al sasaran pengembangan skill
2,PT Hacktivate Teknologi Indonesia,"Gedung Aquarius Lt. 1&2, Jl. Sultan Iskandar M...",Memahami serta mengimplementasikan library hin...,memahami serta mengimplementasikan library hin...
3,PT Ruang Raya Indonesia,"Jl. Dr. Saharjo No.161, Manggarai Selatan, Teb...","Final Project,software engineering : menerapka...",final project software engineering menerapkan...
4,PT BISA ARTIFISIAL INDONESIA,JLN. IR H JUANDA NO. 84 BANDUNG,Mengenal teknik dan langkah dalam manajemen pr...,mengenal teknik dan langkah dalam manajemen pr...


## 4. Modul Pembelajaran (Setelah Preprocessing)

In [34]:
# Deskripsi kedua (Setelah preprocessing)
def print_description_clean(index):
    example = df[df.index == index][['modulpembelajaran_clean', 'Nama Perusahaan','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama Perusahaan :', example[1])
        print('Alamat:', example[2])

In [35]:
print_description_clean(1)

siklus project al sasaran pengembangan skill 
Nama Perusahaan : PT Orbit Ventura Indonesia
Alamat: Veteran RI Building 15th Floor Unit Z15-002, Plaza Semanggi Jl.Jend.Sudirman Kav. 50, Jakarta Indonesia - 12930


In [36]:
print_description_clean(6)

systems analysis and design communication skills team work time management software engineering
Nama Perusahaan : PT. IlmuKomputerCom Braindevs Sistema
Alamat: Jl. Gatot Subroto Kav. 71-73 Pancoran Jakarta Selatan 12870


In [37]:
print_description(10)

Mampu menggunakan metode SDLC,peserta mempelajari metode SDLC dengan cara mencari sebuah sistem yang open source code, melakukan modifikasi,presentasi sistem yang telah dimodifikasi
Nama Perusahaan : PT Pundi Mas Berjaya
Alamat: Jl. Kw. Industri Tunas 2 No.7D, Belian, Kec. Batam Kota, Kota Batam, Kepulauan Riau 29444


## 5. TF-IDF & Cosine Similarity

In [38]:
df.set_index('Nama Perusahaan', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['modulpembelajaran_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.        , 0.        , 0.02670802, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.02593564, 0.00981292],
       [0.        , 1.        , 0.        , 0.03600946, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.05980121, 0.01968191,
        0.        , 0.00951774, 0.        , 0.03908196, 0.        ,
        0.03076556, 0.01167966],
       [0.02670802, 0.03600946, 0.05980121, 1.        , 0.02743355,
        0.        , 0.04321972, 0.        , 0.01718847, 0.        ,
        0.028191  , 0.0457023 ],
       [0.        , 0.        , 0.01968191, 0.02743355, 1.        ,
        0.        , 0.        , 0.        , 0.01241917, 0.        ,
        0.01566642, 0.02360929],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.01592584, 0.        , 0.        , 0.01561931,
        0.        ,

In [40]:
# Set index utama di kolom 'name'
indices = pd.Series(df.index)
indices[:12]

0                             Adis Amatulia 
1                 PT Orbit Ventura Indonesia
2          PT Hacktivate Teknologi Indonesia
3                    PT Ruang Raya Indonesia
4               PT BISA ARTIFISIAL INDONESIA
5                  PT Mitra Semeru Indonesia
6      PT. IlmuKomputerCom Braindevs Sistema
7     PT Bank Negara Indonesia (Persero) Tbk
8                  PT Zona Edukasi Nusantara
9          PT Telkom Indonesia (Persero) Tbk
10                      PT Pundi Mas Berjaya
11              PT Inamas Sintesis Teknologi
Name: Nama Perusahaan, dtype: object

## 6. Modelling

In [41]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_kpsmerdeka = []
    
    # Mengambil nama hotel berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 10 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:4].index)
    
    for i in top_10_indexes:
        recommended_kpsmerdeka.append(list(df.index)[i])
        
    return recommended_kpsmerdeka

## 7. Prediksi

In [50]:
recommendations('Adis Amatulia ')

['PT Ruang Raya Indonesia',
 'PT Pundi Mas Berjaya',
 'PT Inamas Sintesis Teknologi']

In [43]:
recommendations("PT Ruang Raya Indonesia")

['PT Hacktivate Teknologi Indonesia',
 'PT Inamas Sintesis Teknologi',
 'PT. IlmuKomputerCom Braindevs Sistema']

In [46]:
recommendations("PT. IlmuKomputerCom Braindevs Sistema")

['PT Ruang Raya Indonesia',
 'PT Telkom Indonesia (Persero) Tbk',
 'PT Bank Negara Indonesia (Persero) Tbk']

# Selesai...